*** Program to Solve Ordinary Sudoku Puzzles ***

    by Danel Sanchez Tarrago

An ordinary Sudoku is a 9 x 9 grid that needs to be filled with numbers in such a way that each column, each row, and each of the 3 x 3 subgrids or blocks that compose the grid contain all of the digits from 1 to 9. The puzzle starts with a partial filled grid that needs to be completed by the player. 

The algorithm introduced here to solve sudoku puzzles is a hybrid method that combines logic deduction with a backtracking algorithm. Whenever possible it tries to infere new sudoku numbers by applying deduction rules, that are based on the sudoku constrains, to the numbers already in the grid. When deduction is not possible, a backtraking algorithm searches the space of possible solutions starting with the most feasible cell. When the value of a new cell is found by the backtracking algorithm, the deduction algorithm is applied again to the updated grid to maximize the opportunities to find new sudoku numbers. 

Most sudoku puzzles can be solve by the algorithm within 7 to 10 recursive backtracking levels. The hardest sudoku puzzles are solve in less than 15 recursive backtracking levels.

Numpy arrays are used to store and manipulate Sudokus' data

In [1]:
import numpy as np

An array of strings is used to represent a Sudoku. The array has 9 strings, each one being a 9 character length string representing a row in the Sudoku. Each character in the string can be a number betwen 1-9 or a '-' representing
a blank space in the sudoku. This textual representation of a sudoku is intended to facilitate data input. 

The following matrix (su0) represents a totally empty sudoku. It is used as a template, to facilite the edition of new sudokus. 

In [2]:
su0 = np.array(['---------',
                '---------',
                '---------',
                '---------',
                '---------',
                '---------',
                '---------',
                '---------',
                '---------'])

The following are some sudoku samples. They are sorted in complexity order: su1 is the easiest, and su7 is the hardest to solve. They can be used to test the program capabilities.

In [3]:
su1 = np.array(['53--7----',
                '6--195---',
                '-98----6-',
                '8---6---3',
                '4--8-3--1',
                '7---2---6',
                '-6----28-',
                '---419--5',
                '----8--79'])

In [4]:
su2= np.array(['-------18',
                '7----9265',
                '8-451---9',
                '-45872--1',
                '---------',
                '9--36154-',
                '2---947-3',
                '4571----2',
                '36-------'])

In [5]:
su3 = np.array(['----751-2',
                '----1-6--',
                '-8-----47',
                '------4-6',
                '15-642-98',
                '6-3------',
                '42-----7-',
                '--5-2----',
                '7-945----'])

In [6]:
su4 = np.array(['-1--9-28-',
                '--8--2--6',
                '--7-5--34',
                '-----1---',
                '---985---',
                '---3-----',
                '98--6-4--',
                '3--8--6--',
                '-54-3--1-'])

In [7]:
su5 = np.array(['2----5-4-',
                '4---26---',
                '5---9-8--',
                '------9-3',
                '-68---17-',
                '1-3------',
                '--4-5---6',
                '---16---5',
                '-1-3----8'])

In [8]:
su6 = np.array(['---1----4',
                '-6-45---9',
                '3----2---',
                '19--3-6--',
                '--2---1--',
                '--5-7--28',
                '---8----5',
                '7---29-8-',
                '9----7---'])

In [9]:
su7 = np.array(['-461-----',
                '------95-',
                '7---45--8',
                '9---567--',
                '---------',
                '--179---5',
                '4--91---6',
                '-93------',
                '-----842-'])

The getmatrix() function gets a textual representation of a sudoku (txtmat) and converts it in a numeric matrix. Blank spaces are represented as zeros in the output matrix.

In [10]:
def getmatrix(txtmat):
    arr = np.array([np.array(list(row)) for row in txtmat])
    arr[arr == '-'] = 0
    return arr.astype(int)

In [25]:
m7 = getmatrix(su7)
m7

array([[0, 4, 6, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 9, 5, 0],
       [7, 0, 0, 0, 4, 5, 0, 0, 8],
       [9, 0, 0, 0, 5, 6, 7, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 7, 9, 0, 0, 0, 5],
       [4, 0, 0, 9, 1, 0, 0, 0, 6],
       [0, 9, 3, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 8, 4, 2, 0]])

The gettraces() function returns a 9x9 matrix where each cell contains the list of possible values for the corresponding cell in the sudoku grid. A possible value for a cell is one in the range 1 to 9, that is not currently present in the row, nor in the column, and neither in the subgrid the cell belongs to.


In [11]:
def gettraces(arr):
    candidates = np.empty((3, 3), dtype=list)
    traces = np.empty((9, 9), dtype=list)
    for r in range(3):
        for c in range(3):
            block = arr[r*3:r*3+3, c*3:c*3+3].ravel()
            candidates[r, c] = np.setdiff1d(np.arange(1, 10), block[np.nonzero(block)]).tolist()
    for ri in range(9):
        hornonzero = arr[ri,:][np.nonzero(arr[ri,:])]
        can_row = int(ri/3)
        for ci in range(9):
            if arr[ri, ci] == 0:
                vernonzero = arr[:,ci][np.nonzero(arr[:,ci])]
                can_col = int(ci/3)
                hor = np.setdiff1d(candidates[can_row, can_col], hornonzero)
                ver = np.setdiff1d(candidates[can_row, can_col], vernonzero)
                traces[ri, ci] = np.intersect1d(hor, ver).tolist()
            else:
                traces[ri, ci] = [arr[ri, ci]]
    return traces

In [27]:
tr7 = gettraces(m7)
tr7

array([[list([2, 3, 5, 8]), list([4]), list([6]), list([1]),
        list([2, 3, 7, 8]), list([2, 3, 7, 9]), list([2, 3]),
        list([3, 7]), list([2, 3, 7])],
       [list([1, 2, 3, 8]), list([1, 2, 3, 8]), list([2, 8]),
        list([2, 3, 6, 8]), list([2, 3, 6, 7, 8]), list([2, 3, 7]),
        list([9]), list([5]), list([1, 2, 3, 4, 7])],
       [list([7]), list([1, 2, 3]), list([2, 9]), list([2, 3, 6]),
        list([4]), list([5]), list([1, 2, 3, 6]), list([1, 3, 6]),
        list([8])],
       [list([9]), list([2, 3, 8]), list([2, 4, 8]), list([2, 3, 4, 8]),
        list([5]), list([6]), list([7]), list([1, 3, 4, 8]),
        list([1, 2, 3, 4])],
       [list([2, 3, 5, 6, 8]), list([2, 3, 5, 6, 7, 8]),
        list([2, 4, 5, 7, 8]), list([2, 3, 4, 8]), list([2, 3, 8]),
        list([1, 2, 3, 4]), list([1, 2, 3, 6, 8]),
        list([1, 3, 4, 6, 8, 9]), list([1, 2, 3, 4, 9])],
       [list([2, 3, 6, 8]), list([2, 3, 6, 8]), list([1]), list([7]),
        list([9]), list([2, 3, 4

If a cell in the trace Matrix has only one possible value, that possible value is the correct value for the corresponding cell in the sudoku grid. The derive() function receives a trace matrix as a parameter, and returns a sudoku grid filled with a number corresponding to every single-element list in the trace matrix. 


In [12]:
def derive(tr):
    R = np.zeros((9,9), dtype=int)
    for ri in range(9):
        for ci in range(9):
            if len(tr[ri, ci]) == 1:
                R[ri, ci] = tr[ri, ci][0]
    return R

The solvesteps() function informs the number of cells that remains to be solve.

In [13]:
def solvesteps(m):
    return m.shape[0] * m.shape[1] - np.count_nonzero(m)

The is_ok() function determines if the array arr is a valid sudoku array, i.e., if it complies with the sudoku rules.

In [15]:
def is_ok(arr):
    for row in arr:
        nonzero_vals = row[np.nonzero(row)]
        if nonzero_vals.size > np.unique(nonzero_vals).size: return False
    for col_ind in range(9):
        nonzero_vals = arr[:,col_ind][np.nonzero(arr[:,col_ind])]
        if nonzero_vals.size > np.unique(nonzero_vals).size: return False
    return True

The reduce() function calls gettraces() interatively, each time using as parameter the sudoku grid derived from the previous gettraces()'s outcome. In this way, each new sudoku cell number discovered by gettraces() is used in a next step to reduce the number of possible values for each cell and eventually discover new sudoku cell numbers. The iteration ends when no new sudoku number is discover.

In [14]:
def reduce(arr, trace=False):
    a = np.array(arr, dtype=int)
    prev_steps = a.shape[0] * a.shape[1]
    steps = solvesteps(a)
    while steps < prev_steps:
        tr = gettraces(a)
        a = derive(tr)
        prev_steps = steps
        steps = solvesteps(a)
    if trace:
        return a, tr
    else:
        return a

The opportunities() function computes a 9 x 9 matrix where each cell x in this matrix contains the sum of possible values of all the cells in the row, the column, and the subgrid the cell x belongs to. The lesser the sum the easier to find the sudoku number corresponding to that cell because it that means there is a smaller number of solutions to try on. 

In [16]:
def opportunities(trace):
    lenarr = np.array([len(w) for w in trace.flat]).reshape(9, 9)
    opp = np.full_like(lenarr, 1000, dtype=int)
    col_sum = np.sum(lenarr, axis=0)
    row_sum = np.sum(lenarr, axis=1)
    block_sum = np.empty((3, 3), dtype=int)
    for r in range(3):
        for c in range(3):
            block = lenarr[r*3:r*3+3, c*3:c*3+3].ravel()
            block_sum[r, c] = np.sum(block)
    for ri in range(9):
        block_row = int(ri/3)
        for ci in range(9):
            block_col = int(ci/3)
            if lenarr[ri, ci] > 1: opp[ri, ci] = col_sum[ci] + row_sum[ri] + block_sum[block_row, block_col]
    return opp

The bestchoice() function returns the coordinates of the cell that has a smaller number of possibilities to try on. This cell is the best place to beginning the exploration of the solution space.

In [17]:
def bestchoice(trace):
    opp = opportunities(trace)
    n = opp.argmin()
    row = int(n/9)
    col = n%9
    return (row, col)

solvematrix() is a recursive function that explores the solution space using a best-first aproach. It first tries to deduce as many sudoku numbers as possible by using the function reduce(). If the resulting matrix is not a valid solution, the function backtracks and look for another solution. If all the cells have been found, the function returns with the right solution. In other case, the function continues by choosing the best cell and cycles through the list of possible values of that cell. For each value, a copy of the sudoku matrix is made and the value inserted on it. Then, solvematrix() function is called recursively on the updated copy of the matrix. In the next iteration of the recursive function, the new added value is used to deduce new sudoku numbers.

In [18]:
def solvematrix(matrix, step, maxdepth):
    reduced, traced = reduce(matrix, trace=True)
    if not is_ok(reduced): return False, None
    if solvesteps(reduced) == 0: return True, reduced
    else:
        if step < maxdepth:
            (row, col) = bestchoice(traced)
            for w in traced[row, col]:
                nextmatrix = reduced.copy()
                nextmatrix[row, col] = w
                result, solution = solvematrix(nextmatrix, step+1, maxdepth)
                if result: return True, solution
        return False, None

The solve() function is used as the interface with the algorithm. It is the function to be called by the user to solve the sodoku puzzle. It sets up and starts the recursive function solvematrix(). 

In [19]:
def solve(sudoku, maxdepth=15):
    matrix = getmatrix(sudoku)
    result, solution = solvematrix(matrix, 1, maxdepth)
    if result:
        return solution
    else:
        return None

In [20]:
solve(su7)

array([[5, 4, 6, 1, 8, 9, 2, 3, 7],
       [3, 1, 8, 6, 7, 2, 9, 5, 4],
       [7, 2, 9, 3, 4, 5, 1, 6, 8],
       [9, 3, 4, 8, 5, 6, 7, 1, 2],
       [6, 7, 5, 4, 2, 1, 8, 9, 3],
       [2, 8, 1, 7, 9, 3, 6, 4, 5],
       [4, 5, 2, 9, 1, 7, 3, 8, 6],
       [8, 9, 3, 2, 6, 4, 5, 7, 1],
       [1, 6, 7, 5, 3, 8, 4, 2, 9]])